# 🍁 C1 & C3 Capital

This code below has the goal to quickly attribute the C1 capital factor to every security in CLL / CLA's portfolios.  
Then we will compute C3 capital.

For C1 Capital our work is based on OSFI regulation.  
You can access the regulations using this website: https://www.osfi-bsif.gc.ca/sites/default/files/2023-11/licat24.pdf 

## 👩‍💻 Import the processed file - Securities tab

The goal here is to import the Security dataframe from the Securities file

In [107]:
import pandas as pd

# Define the path to your file
file_path = r'C:/Users/bernier/Development/Ogma/CLL/Securities/20231231_securities.xlsx'

# Load the "Securities" sheet without specifying the header
temp_df = pd.read_excel(file_path, sheet_name='Securities', header=None, engine='openpyxl')

# Find the first non-empty row in column A
first_non_empty_row = temp_df[0].first_valid_index()

# Load the "Securities" sheet again using the first non-empty row as the header
ProcessedSecuritiesDF = pd.read_excel(file_path, sheet_name='Securities', header=first_non_empty_row, engine='openpyxl')

# Display the first few rows of the DataFrame
#ProcessedSecuritiesDF.head()

## 💸 Import the processed file - CashFlows tab

The goal here is to import the CashFlows dataframe from the Securities file

In [32]:
import pandas as pd

# Define the path to your file
file_path = r'C:/Users/bernier/Development/Ogma/CLL/Securities/20231231_securities.xlsx'

# Load the "Securities" sheet without specifying the header
temp_df = pd.read_excel(file_path, sheet_name='CashFlows', header=None, engine='openpyxl')

# Find the first non-empty row in column A
first_period_row = temp_df[temp_df[0] == "Period"].index[0]

# Load the "Securities" sheet again using the first non-empty row as the header
ProcessedCFsDF = pd.read_excel(file_path, sheet_name='CashFlows', header=first_period_row, engine='openpyxl')

# Display the first few rows of the DataFrame
#ProcessedCFsDF.head()

## 📌 C1 factor

We import the Canadian OSFI C1 factor Credit Risk Factors by Rating and Effective Maturity (in years)  
It will built has a 3-columns table. The first one would be the Rating, the second is the effective maturity and the third will be factor

In [108]:
# Define the path to your file
file_path = r'C:/Users/bernier/Development/C1Factors.xlsx'

# Load the "Securities" sheet again using the first non-empty row as the header
C1FactorsDF = pd.read_excel(file_path, engine='openpyxl')

# Display the first few rows of the DataFrame
#C1FactorsDF.head()

## ⚙️ C1 asset category attribution process  

The C1 attribution process determines how each security should be treated.  
It has 8 categories:  
- Entities eligible for a 0% factor: see 3.1.4. section of the OSFI regulation  
- Mortgages that are guaranteed by Canada Mortgage and Housing Corporation (CMHC): see 3.1.6. section of the OSFI regulation  
- Residential mortgage loans: see 3.1.6. section of the OSFI regulation    
- Commercial mortgage loans: see 3.1.6. section of the OSFI regulation  
- Leases: see 3.1.9. section of the OSFI regulation  
- Equity: see section 5.2.1 of the OSFI regulation   
- *Property -- to be completed*
- Debt: see section 3.1.2 of the OSFI regulation

We will numerate them so thet will easen the process

### 🧩 C1 Categories

In [118]:

credit_group_0 = [
    "Canada",
    "Province of Quebec",
    "International Bank For Reconstruction & Development",
    "European Stability Mechanism",
    "European Investment Bank",
    "Council Of Europe Development",
    "The European Bank for Reconstruction and Development",
    "States of Jersey",
    "Republic of Poland",
    "Republic of France",
    "Republic of Finland",
    "Norway (Kingdom of)",
    "Nordic Investment Bank",
    "Kingdom of the Netherlands",
    "Kingdom of Sweden",
    "Kingdom Of Spain",
    "Kingdom of Norway",
    "Kingdom of Denmark",
    "Kingdom of Belgium",
    "Japan",
    "Isle of Man",
    "Inter-American Development Bank",
    "Government of United Kingdom",
    "Germany (Federal Republic of)",
    "Federal Republic of Germany",
    "Emirate of Dubai United Arab Emirates",
    "Asian Development Bank",
    "African Development Bank",
]

calm_asset_type_Bonds = [
    "Bond - IL", 
    "Bond - NL", 
    "FRN",
]

calm_asset_type_Mortgages = [
    "Mortgage - NL",
    "Mortgage - IL",
]

calm_asset_type_ERM = [
    "ERM",
]

calm_asset_type_Leases = [
    "Fin Lease - NL",
    "Fin Arrangement - IL",
]

calm_asset_type_Property = [
    "Property - NL",
    "Property - IL",
    "Property - IL Backing NL",
]

calm_asset_type_SWAPS = [
    "Inflation SWAP",
]

# Function to assign C1AssetCategory based on the mappings
def assign_c1_asset_category(row):
    if row['CreditGroupName'] in credit_group_0:
        return 0
    elif row['CALMAssetType'] in calm_asset_type_Bonds:
        return 1
    elif row['CALMAssetType'] in calm_asset_type_Mortgages:
        return 2
    elif row['CALMAssetType'] in calm_asset_type_ERM:
        return 3
    elif row['CALMAssetType'] in calm_asset_type_Leases:
        return 4
    elif row['CALMAssetType'] in calm_asset_type_Property:
        return 5
    elif row['CALMAssetType'] in calm_asset_type_SWAPS:
        return 6
    else:
        return None
        
# Apply the function to create the new column
ProcessedSecuritiesDF['C1AssetCategory'] = ProcessedSecuritiesDF.apply(assign_c1_asset_category, axis=1)

# Set display option to show all columns
pd.set_option('display.max_columns', None)

# Display the first few rows of the updated DataFrame
#print(ProcessedSecuritiesDF.head(50))

### 🎯 Calculate C1 factor

In [126]:
# Normalize the Rating column
rating_mapping = {
    "AA+": "AA",
    "AA-": "AA",
    "A+": "A",
    "A-": "A",
    "BBB+": "BBB",
    "BBB-": "BBB",
    "BB+": "BB",
    "BB-": "BB",
    "B+": "B",
    "B-": "B",
    "DDD": "Below_B",
    "CCC": "Below_B",
    "CCC-": "Below_B",
}
ProcessedSecuritiesDF["NormalizedRating"] = ProcessedSecuritiesDF["Rating"].replace(rating_mapping)
#print("First few rows after rating normalization:")

#print(C1FactorsDF[['EffectiveMaturity', 'Rating']].head())
# Round the ModDuration column

ProcessedSecuritiesDF["RoundedModDuration"] = ProcessedSecuritiesDF["ModDuration"].round().astype(int)
#print(ProcessedSecuritiesDF[['Rating', 'NormalizedRating','ModDuration','RoundedModDuration']].head())
# Filter rows with valid ratings
valid_ratings = C1FactorsDF['Rating'].unique()
ProcessedSecuritiesDF = ProcessedSecuritiesDF[ProcessedSecuritiesDF['NormalizedRating'].isin(valid_ratings)]

# Debugging: Check unique values and data types before merging
# FOR TESTING data type need  to be the same - Can be deleted -
#print("Unique values in ProcessedSecuritiesDF['NormalizedRating']:", ProcessedSecuritiesDF['NormalizedRating'].unique())
#print("Unique values in ProcessedSecuritiesDF['RoundedModDuration']:", ProcessedSecuritiesDF['RoundedModDuration'].unique())
#print("Unique values in C1FactorsDF['Rating']:", C1FactorsDF['Rating'].unique())
#print("Unique values in C1FactorsDF['EffectiveMaturity']:", C1FactorsDF['EffectiveMaturity'].unique())

#print("Data types of merging columns:")
#print("ProcessedSecuritiesDF['NormalizedRating']:", ProcessedSecuritiesDF['NormalizedRating'].dtype)
#print("ProcessedSecuritiesDF['RoundedModDuration']:", ProcessedSecuritiesDF['RoundedModDuration'].dtype)
#print("C1FactorsDF['Rating']:", C1FactorsDF['Rating'].dtype)
#print("C1FactorsDF['EffectiveMaturity']:", C1FactorsDF['EffectiveMaturity'].dtype)

# Ensure the data types of merging columns match
ProcessedSecuritiesDF['NormalizedRating'] = ProcessedSecuritiesDF['NormalizedRating'].astype(str)
ProcessedSecuritiesDF['RoundedModDuration'] = ProcessedSecuritiesDF['RoundedModDuration'].astype(int)
C1FactorsDF['Rating'] = C1FactorsDF['Rating'].astype(str)
C1FactorsDF['EffectiveMaturity'] = C1FactorsDF['EffectiveMaturity'].astype(int)

# Rename conflicting columns in ProcessedSecuritiesDF before merging
conflicting_columns = ['Rating', 'EffectiveMaturity', 'C1Factor']
ProcessedSecuritiesDF.rename(columns={col: f"{col}_sec" for col in conflicting_columns}, inplace=True)

# Merge ProcessedSecuritiesDF with C1FactorsDF on NormalizedRating and RoundedModDuration
ProcessedSecuritiesDF = pd.merge(
    ProcessedSecuritiesDF,
    C1FactorsDF,
    left_on=["NormalizedRating", "RoundedModDuration"],
    right_on=["Rating", "EffectiveMaturity"],
    how="left",
    suffixes=('_sec', '_factor')  # Adding suffixes to avoid column name conflicts
)


# Function to assign C1AssetCategory based on the mappings
def assign_c1_factor(row):
    if row["C1AssetCategory"] == 0:
        return 0
    elif row["C1AssetCategory"] == 1:
        return row['C1Factor']
    elif row["C1AssetCategory"] == 2:
        return 0.06
    elif row["C1AssetCategory"] == 3:
        return 0.06
    elif row["C1AssetCategory"] == 4:
        return 0.06
    elif row["C1AssetCategory"] == 5:
        return 0.3
    elif row["C1AssetCategory"] == 6:
        return 0
    else:
        return None


# Apply the function to create the new column
ProcessedSecuritiesDF["C1"] = ProcessedSecuritiesDF.apply(assign_c1_factor, axis=1)

# Set display option to show all columns
pd.set_option("display.max_columns", None)

# Display the first few rows of the updated DataFrame
#print(ProcessedSecuritiesDF.head(50))